In [67]:
from google.colab import files

files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [68]:
! kaggle datasets download -d vladislavkovalev/itadatahackday2
! kaggle datasets download -d vladislavkovalev/itadatahack2-selfish

itadatahackday2.zip: Skipping, found more recently modified local copy (use --force to force download)
 99% 47.0M/47.4M [00:02<00:00, 25.5MB/s]
100% 47.4M/47.4M [00:02<00:00, 16.9MB/s]


In [3]:
! unzip itadatahackday2.zip -d data

Archive:  itadatahackday2.zip
  inflating: data/new_test.csv       
  inflating: data/train_set.csv      


In [69]:
! unzip itadatahack2-selfish.zip -d data

Archive:  itadatahack2-selfish.zip
  inflating: data/selfish.csv        


In [1]:
!pip install -q transformers==4.17.0
!pip install -q datasets==2.3.2
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import re
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

In [5]:
data_test = pd.read_csv('/content/data/new_test.csv')
data_train = pd.read_csv('/content/data/train_set.csv')
data_train = data_train.drop_duplicates()
data_train.columns

Index(['CELEX_ID', 'Text', 'Directory code', 'Citations'], dtype='object')

In [18]:
data_selfish = pd.read_csv('/content/data/selfish.csv')
data_selfish = data_selfish.dropna()
data_selfish.shape

(4877, 5)

In [6]:
id2label = {idx: value for idx, value in enumerate(sorted(data_train['Directory code'].unique()))}
label2id = {value: idx  for idx, value in enumerate(sorted(data_train['Directory code'].unique()))}

In [21]:
nltk.download('stopwords')
nltk.download('wordnet')
tqdm.pandas()
stop_words = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 5000000

def preprocess_text(text):
    text = ' '.join([word.lower() for word in text.split() if word not in stop_words])
    text = re.sub("^[A-Za-z0-9]*$", "", text)
    doc = nlp(text)
    filtered_tokens = [token.lemma_ for token in doc ]
    return ' '.join(filtered_tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
data_train['Text'] = data_train['Text'].progress_apply(preprocess_text)
data_test['Text'] = data_test['Text'].progress_apply(preprocess_text)

100%|██████████| 2486/2486 [12:07<00:00,  3.42it/s]


In [22]:
data_selfish['Text'] = data_selfish['Text'].progress_apply(preprocess_text)

100%|██████████| 4877/4877 [47:48<00:00,  1.70it/s]


In [23]:
data_train = pd.concat([data_train, data_selfish])

In [30]:
data_train = data_train.drop(columns='Unnamed: 0').reset_index(drop=True)

In [31]:
data_train.to_csv('lemmed_train_selfish.csv')
data_test.to_csv('lemmed_test.csv')

In [5]:
data_train = pd.read_csv('lemmed_train.csv')
data_test = pd.read_csv('lemmed_test.csv')

In [32]:
data_train['Citations']  = data_train['Citations'].apply(lambda x:
                                                      ', '.join(x.replace('[', '')\
                                                                .replace(']', '')\
                                                                .replace("'", '')
                                                                .split(',')))

In [33]:
data_train['Text'] = data_train['Citations'] + " " + data_train['Text']

In [34]:
data_train = data_train[['CELEX_ID', 'Text', 'Directory code']]
data_train['Text'] = data_train.Text.astype('str')

<ipython-input-34-3af7668d198c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['Text'] = data_train.Text.astype('str')


In [35]:
def numbers_to_classes(l):
  zero_cl = [0.0] * len(data_train['labels'].unique())
  zero_cl[int(l)] = 1.0
  return np.array(zero_cl, dtype=np.float)

In [36]:
data_train

,CELEX_ID,Text,Directory code
0,32012D0012(01),"OJL341, 22.12.2011, p.65, OJL331, 14.12.2011,...",10-30
1,52018DP0022,"OJL353, 31.12.2008, p.1, OJL208, 2.8.2016, p....",01-40
2,32020D1214(02),"OJC315I, 23.9.2020, p.5, OJL141, 27.5.2011, p...",01-40
3,32010D1007(01),"OJL8, 12.1.2001, p.1, OJL196, 24.7.2008, p.1,...",13-20
4,32019D1742,"OJL128, 9.5.2013, p.1, OJL96, 31.3.2004, p.1,...",07-40
...,...,...,...
14813,42020Z10201,cooperation agreement the european central ba...,01-40
14814,42021D0323,decision eu 2021323 of the represent ative of...,01-40
14815,42021D0324,decision eu 2021324 of the represent ative of...,01-40
14816,42021D1462,decision eu 20211462 of the represent ative o...,01-40


In [37]:
data_train = data_train.rename(
    {"CELEX_ID": "celex_id", "Text": "text", "Directory code": 'labels'},
    axis=1)[['text', 'labels']]
data_train.reset_index(inplace=True)

In [38]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]

  # encode them
  encoding = tokenizer(text, padding="max_length",
                       truncation=True, max_length=500)

  # add labels
  labels_matrix = []
  for r in examples["labels"]:
    labels_matrix.append(np.array(numbers_to_classes(label2id[r]), dtype=float))
  # print(labels_matrix)
  encoding["labels"] = np.array(labels_matrix, dtype=float)

  return encoding

In [39]:
hg_dataset = Dataset(pa.Table.from_pandas(data_train))
encoded_dataset = hg_dataset.map(preprocess_data, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

<ipython-input-35-03c2f3176ba2>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.array(zero_cl, dtype=np.float)


In [43]:
encoded_dataset = encoded_dataset.train_test_split(test_size=0.17)

In [44]:
encoded_dataset.set_format("torch")
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12298
    })
    test: Dataset({
        features: ['index', 'text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2520
    })
})

In [45]:
model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(data_train.labels.unique()),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [46]:
batch_size = 16
metric_name = "f1"

In [47]:
args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=12,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name
)

In [48]:
def multi_label_metrics(predictions, labels):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))

    y_pred = np.zeros(probs.shape)
    num_rows = probs.size(0)
    y_pred[range(num_rows), np.argmax(probs, axis=1)] = 1

    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [49]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: index, text. If index, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12298
  Num Epochs = 12
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9228


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.141600,0.052089,0.255952,0.623650,0.255952
2,0.041100,0.034978,0.651587,0.823768,0.651587
3,0.031400,0.024217,0.777381,0.887396,0.777381
4,0.020300,0.019220,0.811508,0.904658,0.811508
5,0.017000,0.016446,0.833730,0.915898,0.833730
6,0.013300,0.014977,0.843651,0.920916,0.843651
7,0.011500,0.013855,0.850397,0.924329,0.850397
8,0.009600,0.013015,0.855556,0.926938,0.855556
9,0.008700,0.012625,0.857143,0.927741,0.857143


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: index, text. If index, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2520
  Batch size = 16
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-769
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-769/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-769/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-769/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-769/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: index, text. If index, text are not expected by `BertForSequenceC

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.141600,0.052089,0.255952,0.623650,0.255952
2,0.041100,0.034978,0.651587,0.823768,0.651587
3,0.031400,0.024217,0.777381,0.887396,0.777381
4,0.020300,0.019220,0.811508,0.904658,0.811508
5,0.017000,0.016446,0.833730,0.915898,0.833730
6,0.013300,0.014977,0.843651,0.920916,0.843651
7,0.011500,0.013855,0.850397,0.924329,0.850397
8,0.009600,0.013015,0.855556,0.926938,0.855556
9,0.008700,0.012625,0.857143,0.927741,0.857143
10,0.007400,0.012189,0.863889,0.931153,0.863889


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: index, text. If index, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2520
  Batch size = 16
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-7690
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-7690/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-7690/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-7690/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-7690/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: index, text. If index, text are not expected by `BertForSequ

In [ ]:
def predict_label(text):
    encoding = tokenizer(text, padding="max_length",
                          truncation=True, max_length=300, return_tensors="pt")
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = model(**encoding)
    logits = outputs.logits
    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    predictions = np.zeros(probs.shape)
    num_rows = probs.size(0)

    return id2label[probs.argmax() + 1]

In [ ]:
del(data_test['Unnamed: 0'])
del(data_test['CELEX_ID'])
data_test['Citations'] \
    = data_test['Citations'].apply(lambda x: ', '.join(x.replace('[', '').replace(']', '').split(',')))
data_test['Text'] = data_test['Citations'].str.replace("'", '') + " [SEP] " + data_test['Text']
data_test.rename({"CELEX_ID": "celex_id", 'Text': 'text'}, axis=1, inplace=True)
data_test = data_test[['text']]
data_test = data_test.reset_index(drop=True)
hg_dataset = Dataset(pa.Table.from_pandas(data_test))

In [ ]:
labels = [int(predict_label(text=text['text'])) for text in hg_dataset]
labels = pd.DataFrame(labels)
labels.to_csv('test_new_answ.csv')